# LLMs prompts

## <span style="color: red;">Warning!</span>

The following materials are for data review purposes only. The results of the prompts **should not be used in scientific work under any circumstances**. The prompts should only be used for preliminary data review. Final statistics and data analysis should be performed by a professional statistician. 

## Data overview

**[Heart Disease Cleveland](https://www.kaggle.com/datasets/ritwikb3/heart-disease-cleveland/data)**

The dataset is the Cleveland Heart Disease dataset taken from the UCI repository. The dataset consists of 303 individuals’ data.


[Licence:CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

1. **Age**: Patients Age in years 
2. **Sex**: Gender (Male : 1; Female : 0) 
3. **cp**: Type of chest pain experienced by patient. This term categorized into 4 category.
    - 0 typical angina,
    - 1 atypical angina,
    - 2 non-anginal pain,
    - 3 asymptomatic 
4. **trestbps**: patient's level of blood pressure at resting mode in mm/HG 
5. **chol**: Serum cholesterol in mg/dl 
6. **fbs**: Blood sugar levels on fasting > 120 mg/dl represents as 1 in case of true and 0 as false 
7. **restecg**: Result of electrocardiogram while at rest are represented in 3 distinct values
   - 0 : Normal
   - 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
   - 2: showing probable or definite left ventricular hypertrophyby Estes' criteria 
8. **thalach**: Maximum heart rate achieved 
9. **exang**: Angina induced by exercise 0 depicting NO 1 depicting Yes
10. **oldpeak**: Exercise induced ST-depression in relative with the state of rest 
11. **slope**: ST segment measured in terms of slope during peak exercise
    - 0: up sloping;
    - 1: flat;
    - 2: down sloping
14. **ca**: The number of major vessels (0–3)
15. **thal**: A blood disorder called thalassemia
    - 0: NULL
    - 1: normal blood flow
    - 2: fixed defect (no blood flow in some part of the heart)
    - 3: reversible defect (a blood flow is observed but it is not normal
16. ***target***: It is the target variable which we have to predict. 1 means patient is suffering from heart disease and 0 means patient is normal.

## Data loading

**PROMPT: I need to load a dataset from given path to pandas'../data/Heart_disease_cleveland_new.csv'. Plase assign it to "df" variable**. 

**PROMPT: Can I see table in a nicer outline?**

**PROMPT: What types of data do I have in my dataset? Use "display" for presenting tables**

## Data cleaning

**PROMPT: I see that "chol" should be a number not an "object" how to check if there is no some accidently placed string in that and other columns where "object" type is shown**

**PROMPT: How can I replace "unknown" by an empty value?**

**PROMPT: I replaced, but "chol" is still an "object" in my dataframe** 

**PROPMPT: How to discover outliers in my dataset?**

**PROMPT: I see that maximum velues for "chol" and "thalach" are outliers as well as minimum value for "target". How to replace them with an empty value?**

**PROMPT: How many missing values are in my dataset?**

**PROMPT: How to remove all records that contain at least one empty value?**

**PROMPT: How to check if there any duplicates?**

## Basic statistics

**PROMPT: How to obtain column names containing continuous and categorical values?**

**PROMPT: How to display histograms for continuous data?**

**PROMPT: How to get basic statistic for continuous data?**

**PROMPT: How to visualize discrete columns?**

**PROMPT: How to get basic statistics for categorical variables?**

**PROMPT: Can I get a table showing the numbers and percentages for each categorical variable?**

**PROMPT: Can I see correlation matrix for my data?**

**PROMPT: I do not see "sex" in the list (Male, Female), how to include them?**

## Statistics on groups

**PROMPT: Can you show basic statistics per sex?**

**PROMPT: Can you show a plot of "chol" vs "age" regarding "target"?**

## Statistical tests

**PROMPT: Can you show me how to check if there is statistical significance between men and women for each variable?**

**PROMPT: How to create two tables containing general statistics: total, men, women. Number and percentage for categorical variables, median and IQR range for continuous variables? Calculate the “p-value” in each table. All floating point numbers should be rounded to three decimal places.**

**PROMPT: How to make a "p" correction? Justify your choice of method.**

## Discovering risk factors

**PROMPT: How to discover risk factors for my "target"?**

**PROMPT: How to discover feature importance regarding "target"?**